In [1]:
import os
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing as mp
from datetime import datetime
from functools import partial
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.optim import Adam
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
os.environ["CUDA_VISIBLE_DEVICES"] = '2'

In [2]:
trained_model_path = '/workdir/security/home/junjiehuang2468/paper/trained_models_weight/ember/adversarial_training/PGD/'
data_path = "/workdir/security/home/junjiehuang2468/paper/data/ember2018/"
train_data_path = data_path + "malwares/" 
test_data_path = data_path + "test_malwares/" 

In [3]:
CUDA = True if torch.cuda.is_available() else False
NUM_WORKERS = 24
BATCH_SIZE = 20
LEAVE_BIT_NUMBER = 20000
KERNEL_SIZE = 500

In [4]:
trainset = pd.read_csv(data_path + 'train_dataset.csv')
validset = pd.read_csv(data_path + 'valid_dataset.csv')
# testset = pd.read_csv(data_path + 'test_dataset.csv')
# testset = testset.iloc[np.argwhere(testset['labels'].values == 1).squeeze(),:]

In [5]:
class ExeDataset(Dataset): # ExeDataset不是我取的，要記得改
    def __init__(self, malware_names, data_path, labels, leave_bit_num):
        self.malware_names = malware_names
        self.data_path = data_path
        self.labels = labels
        self.leave_bit_num = leave_bit_num

    def __len__(self):
        return len(self.malware_names)

    def __getitem__(self, idx):
        with open(self.data_path + self.malware_names[idx] + '.txt','rb') as fp:
            data = [bit+1 for bit in fp.read()[:self.leave_bit_num]]
            padding = [0]*(self.leave_bit_num-len(data))
            data = data + padding

        return np.array(data), np.array([self.labels[idx]])

In [6]:
train_dataset = ExeDataset(
    trainset["id"].tolist(), 
    train_data_path, 
    trainset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)
valid_dataset = ExeDataset(
    validset["id"].tolist(), 
    train_data_path, 
    validset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)
# test_dataset = ExeDataset(
#     testset["id"].tolist(), 
#     test_data_path, 
#     testset["labels"].tolist(), 
#     LEAVE_BIT_NUMBER
# )

In [7]:
trainloader = DataLoader(
    dataset = train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)
validloader = DataLoader(
    dataset = valid_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)
# testloader = DataLoader(
#     dataset = test_dataset,
#     batch_size = BATCH_SIZE,
#     shuffle = False,
#     num_workers = NUM_WORKERS,
#     pin_memory = True
# )

In [8]:
class Model(nn.Module):
    def __init__(self, data_length = 2e6, kernel_size = 500):
        super().__init__()
        self.embedding = nn.Embedding(257, 8, padding_idx=0)
        self.conv_layer_1 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
        # self.bn_1 = nn.BatchNorm1d(128)
        self.conv_layer_2 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
        self.pool_layer_2 = nn.MaxPool1d(data_length//kernel_size)
        self.fc_layer_3 = nn.Linear(128, 128)
        self.fc_layer_4 = nn.Linear(128, 2)
        
    def forward(self,x):
        embedd_x = self.embedding(x)
        # embedd_x.requires_grad = True
        embedd_x.retain_grad()
        x = embedd_x.transpose(-1,-2)
        x_conv_1 = self.conv_layer_1(x[:,:4,:])
        x_conv_2 = torch.sigmoid(self.conv_layer_2(x[:,4:,:]))
        x = x_conv_1*x_conv_2
        del x_conv_1,x_conv_2
        x = self.pool_layer_2(x).squeeze()
        x = self.fc_layer_3(x)
        x = self.fc_layer_4(x)
        return x,embedd_x

In [9]:
def train_def(model,trainloader,loss_fn,otpim,CUDA=True):
    model.train()
    total_acc = []
    # bar = tqdm(validloader)
    bar = tqdm(trainloader)
    for step, (batch_data,batch_label) in enumerate(bar):
        batch_data = batch_data.cuda() if CUDA else batch_data
        batch_label = batch_label.cuda() if CUDA else batch_label
        batch_label = batch_label.squeeze()

        optim.zero_grad()

        true_label = torch.zeros((len(batch_label),2)).cuda()
        for idx,target in enumerate(batch_label.squeeze()): true_label[idx,target] = 1
        true_label = true_label.cuda() if CUDA else true_label

        temp_total_acc = []
        pred,embedd_x = model(batch_data)
        _, predicted = torch.max(pred, 1)
        acc = (batch_label.cpu().data.numpy() == predicted.cpu().data.numpy()).mean()
        temp_total_acc.append(acc)

        mask = (batch_data == 0).float()
        random_padding = torch.randint_like(input=mask.float(),low=1,high=257) * mask
        batch_data += random_padding.long()

        pred,embedd_x = model(batch_data)
        _, predicted = torch.max(pred, 1)
        acc = (batch_label.cpu().data.numpy() == predicted.cpu().data.numpy()).mean()
        temp_total_acc.append(acc)
        for _ in range(8):
            pred,embedd_x = model(batch_data)
            loss = loss_fn(pred, true_label)
            loss.backward()

            attacked_embedd_x = embedd_x + 0.5*embedd_x.grad.sign()
            all_embedd = model.embedding(torch.arange(start=0,end=257).cuda())

            shape = attacked_embedd_x.shape
            attacked_embedd_x = attacked_embedd_x.reshape((shape[0],shape[1],1,shape[2]))
            attacked_embedd_x = attacked_embedd_x.expand(shape[0],shape[1],257,shape[2])

            results = torch.linalg.norm(attacked_embedd_x - all_embedd,ord=2,dim=-1)
            batch_data = ((1-mask)*batch_data + mask*torch.argmin(results,dim=-1)).long()

            pred,embedd_x = model(batch_data)
            _, predicted = torch.max(pred, 1)
            acc = (batch_label.cpu().data.numpy() == predicted.cpu().data.numpy()).mean()
            temp_total_acc.append(acc)

        optim.zero_grad()
        loss = loss_fn(pred,true_label)
        loss.backward()
        optim.step()

        total_acc.append(temp_total_acc)
        total_acc_str = '[' + ' '.join(map(lambda x: '%.9f'%x,np.mean(total_acc,axis=0))) + ']'
        bar.set_description(total_acc_str)
    
    return model

In [10]:
def valid_def(model,validloader,CUDA=True):
    model.eval()
    total_acc = []
    bar = tqdm(validloader)
    # bar = tqdm(trainloader)
    for step, (batch_data,batch_label) in enumerate(bar):
        batch_data = batch_data.cuda() if CUDA else batch_data
        batch_label = batch_label.cuda() if CUDA else batch_label
        batch_label = batch_label.squeeze()

        optim.zero_grad()

        true_label = torch.zeros((len(batch_label),2)).cuda()
        for idx,target in enumerate(batch_label.squeeze()): true_label[idx,target] = 1
        true_label = true_label.cuda() if CUDA else true_label

        temp_total_acc = []
        pred,embedd_x = model(batch_data)
        _, predicted = torch.max(pred, 1)
        acc = (batch_label.cpu().data.numpy() == predicted.cpu().data.numpy()).mean()
        temp_total_acc.append(acc)
        
        total_acc_str = '[' + ' '.join(map(lambda x: '%.9f'%x,np.mean(temp_total_acc))) + ']'
        bar.set_description(f"test：{acc:4f}, test mean: {np.mean(temp_total_acc):4f}")
    
    return model,np.mean(temp_total_acc)

In [11]:
model = Model(data_length=LEAVE_BIT_NUMBER,kernel_size=KERNEL_SIZE)

ce_loss = nn.CrossEntropyLoss()
optim = Adam(model.parameters())

model = model.cuda() if CUDA else model
ce_loss = ce_loss.cuda() if CUDA else ce_less

In [12]:
time_dir = str(datetime.now())
time_dir = time_dir[:time_dir.rfind(':')]
os.mkdir(f'{trained_model_path}{time_dir}')

In [ ]:
for i in range(20):
    print(i)
    model = train_def(model,trainloader,ce_loss,optim,CUDA)
    model,test_acc = valid_def(model,validloader,CUDA)
    save_path = f'{trained_model_path}{time_dir}/PGD_epoch:{i}_test_acc:{test_acc:.6f}.pt'
    torch.save(model.state_dict(),save_path)

0


[0.764157459 0.764053867 0.755300414 0.750328039 0.746823204 0.743853591 0.741557320 0.740003453 0.738363260 0.736999309]:  12%|█▏        | 2896/24000 [13:37<1:39:10,  3.55it/s]